In [26]:
#Run if you want to autoreload your personal modules on change
import autoreload
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
#Next is needed in azure vm to autoreload modules in cwd
import os
pwd=os.popen("pwd").read().rstrip()
import sys
sys.path.append(pwd)
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import numpy as np
from matplotlib import pyplot as plt
import numpy
import sklearn
import torchvision
import torchvision.transforms as transforms
import os
import argparse
#from models import *
import utils
#from utils import progress_bar
import time
from torchvision import models
import model_actions
import az_manage_proc
import load
import log
import many_inj
import many_inj_ni
#from pytorchfi_c.core import fault_injection as pfi_core
from pytorchfi_ni.core import fault_injection as pfi_core
print("GPU available: ",torch.cuda.is_available())
print("OS: ",sys.platform)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
GPU available:  True
OS:  win32


In [91]:
num_batches = 100
c,w,h=3,32,32

In [22]:
#Set up Test Variables:

#Test batch size
num_batches=100

transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

testset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform_test)

test_batch_size=int(numpy.shape(testset.data)[0]/num_batches)
#test_batch_size=100

testloader = torch.utils.data.DataLoader(
    testset, batch_size=test_batch_size, shuffle=False, num_workers=2)

test_features, test_labels = next(iter(testloader))
#Get num channels, width and height of input data
c,w,h=list(test_features[0].size())
#Batchsize, see load18.py or main.py for batch size
num_batches=int(numpy.shape(testset.data)[0]/test_batch_size)

Files already downloaded and verified


In [4]:
model=model_actions.load("checkpoints/bs_128_sch_plateau/bs_128_sch_plateau_epoch_160")

In [29]:
model_actions.test(model)

(0.8552, 35.54125718772411)

In [92]:
#debug=True? num_batches?
pfi_model = pfi_core(model.eval(),num_batches, input_shape=[c,h,w],layer_types=[torch.nn.Conv2d],use_cuda=torch.cuda.is_available())

In [93]:
many_model = many_inj_ni.many_n_inj(pfi_model,1,min_val=0,max_val=0)

In [94]:
model_actions.test(many_model)

IndexError: list index out of range

In [87]:
%debug

> c:\users\gaboc\desktop\tesis\azure download synchcronization\thesis\pytorchfi_ni\core.py(342)_set_value()
    340         )
    341 
--> 342         layerDim = self.LAYERS_DIM[self.get_curr_layer()]
    343 
    344         logging.info(

[[1, 64, 16, 16], [1, 64, 8, 8], [1, 64, 8, 8], [1, 64, 8, 8], [1, 64, 8, 8], [1, 128, 4, 4], [1, 128, 4, 4], [1, 128, 4, 4], [1, 128, 4, 4], [1, 128, 4, 4], [1, 256, 2, 2], [1, 256, 2, 2], [1, 256, 2, 2], [1, 256, 2, 2], [1, 256, 2, 2], [1, 512, 1, 1], [1, 512, 1, 1], [1, 512, 1, 1], [1, 512, 1, 1], [1, 512, 1, 1]]
20


In [77]:
len(list(list(model.children())[0].children()))

10